# 🚀 Automatic1111 + LoRA (Drive: SD_LoRAs) + Anti-Idle
Notebook mínimo: monta o Google Drive, linka a pasta **SD_LoRAs** ao WebUI, inicia com link público (`--share`) e ativa anti-idle.

In [ ]:
# 1) Montar Google Drive
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# 2) Garantir pasta de LoRAs no Drive e informar caminho
import os
lora_drive_path = '/content/drive/MyDrive/SD_LoRAs'
os.makedirs(lora_drive_path, exist_ok=True)
print('✅ Pasta de LoRAs no Drive:', lora_drive_path)

In [ ]:
# 3) Clonar o WebUI e linkar a pasta de LoRAs do Drive
%cd /content
!git clone https://github.com/AUTOMATIC1111/stable-diffusion-webui /content/stable-diffusion-webui
!mkdir -p /content/stable-diffusion-webui/models
!rm -rf /content/stable-diffusion-webui/models/Lora
!ln -s /content/drive/MyDrive/SD_LoRAs /content/stable-diffusion-webui/models/Lora
print('🔗 LoRAs do Drive linkados em /content/stable-diffusion-webui/models/Lora')

In [ ]:
# 4) Iniciar o WebUI com link público
%cd /content/stable-diffusion-webui
!COMMANDLINE_ARGS="--share --skip-torch-cuda-test --skip-python-version-check --precision full --no-half --enable-insecure-extension-access" \
  python launch.py

In [ ]:
%%javascript
// 5) Anti-idle (evita queda por inatividade)
(function(){
  function clickConnect(){
    const host = document.querySelector('colab-connect-button');
    const btn = host && host.shadowRoot ? host.shadowRoot.querySelector('#connect') : null;
    (btn || document.querySelector('#connect'))?.click();
    console.log('Colab ping');
  }
  setInterval(clickConnect, 60000); // a cada 60s
  console.log('✅ Anti-idle ativo');
})();